In [ ]:
    this is the first line of code inlind with text


    this is a line of code inline with text
LOL what was that singing

In [1]:
````
print('hello')
````

SyntaxError: invalid syntax (3336871262.py, line 1)

markdowwwwn


````
code block
````

In [11]:
import numpy as np
a = np.array([1,0,1])[:,None]
c = 3
print(a)

[[1]
 [0]
 [1]]


In [83]:
from collections import Counter

a = "The Dog jumps over the roof".lower()
b = "The cat jumps on the roof".lower()

vocab = Counter()
for word in a.split(' '):
    vocab[word] += 1
    
print(vocab)
print(len(vocab))

for i,word in enumerate(vocab):
    print(i,vocab[word])

Counter({'the': 2, 'dog': 1, 'jumps': 1, 'over': 1, 'roof': 1})
5
0 2
1 1
2 1
3 1
4 1


In [76]:
vocab = set()
vocab.add("ehllo")
vocab.add('ehllo')
vocab.add("Hello")
vocab.add("hello")
print(vocab)
for i,word in enumerate(vocab):
    print(i, word)
vocab_list = list(vocab)
print(vocab_list)
for i,word in enumerate(vocab_list):
    print(i, word)

{'Hello', 'hello', 'ehllo'}
0 Hello
1 hello
2 ehllo
['Hello', 'hello', 'ehllo']
0 Hello
1 hello
2 ehllo


In [79]:
init_weights = np.zeros([4,1])
print(init_weights)


[[0.]
 [0.]
 [0.]
 [0.]]


In [100]:
class Student:
    def __init__(self,age,name,update_age):
        self.age = age
        self.name = name
        self.update_age(update_age)
    def update_age(self, age):
        self.age = age
        print('Done! Your age is: ', self.age)
student1 = Student(18, 'eric', 19)

Done! Your age is:  19


In [87]:
assert(2==1)

AssertionError: 

In [150]:
import numpy as np

a = np.array([[1,2,3]])
b = np.array([[4],
              [5],
              [6]])
c = 10
print(b.dot(a))

[[ 4  8 12]
 [ 5 10 15]
 [ 6 12 18]]


In [123]:
a = np.random.rand(2,2)
b = np.zeros((2,2))
print(a.dot(10))
print(b)

[[7.70701954 3.82001083]
 [2.41216406 2.20468505]]
[[0. 0.]
 [0. 0.]]


In [135]:
a = "Cute dog"
if ("dog" in a.split(' ')):
    print("something")

something


In [147]:
zeros = np.zeros((1,10))
print(zeros)
print(zeros.size)
a = np.array([1,2,3,4])
print(a)
print(a.size)

zeros[8] += 1

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
10
[1 2 3 4]
4


IndexError: index 8 is out of bounds for axis 0 with size 1